<a href="https://colab.research.google.com/github/byiringiroscar/NLP-Data-science-ML-and-Artificial-Intelligence/blob/main/Scrap_linkedin_profile_with_selenium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

Executing: /tmp/apt-key-gpghome.ATtUqbJCGn/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.iyEUjqnAKi/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.nsD02bJzk0/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
gpg: cannot open '/dev/tty': No such device or address
gpg: [stdout]: write error: Broken pipe
gpg: filter_flush failed on c

In [17]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import json
import time
import os
import requests
from urllib.parse import urlparse, unquote
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from selenium.webdriver.common.by import By
import pandas as pd
import re

chrome_options = Options()


service = Service(executable_path=r'/usr/bin/chromedriver')
chrome_options.add_argument('--disable-blink-features=AutomationControlled')
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')


driver = webdriver.Chrome(service=service, options=chrome_options)

In [3]:
def contains_linkedin(url):
    return "linkedin" in url and url.startswith('https://rw.linkedin.com/in/')


In [4]:
# # Go to the Google homepage
# driver.get("https://www.google.com/")

# # Find the search bar and enter the search term
# key_word = 'linkedin'
# word = 'ishimwe diane'
# search_bar = driver.find_element(By.NAME, 'q')
# search_bar.send_keys(f"{word} {key_word}")
# path = "//input[@type='submit']"

# # Wait for the search button to be clickable
# search_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, path)))

# # Click the search button

# # search_button = driver.find_element(By.XPATH, path)
# search_button.click()

# # Wait for the search results to load
# driver.implicitly_wait(10)
# items = []
# last_height = driver.execute_script("return document.body.scrollHeight")
# itemTargetCount = 100

# scrapped_links = []

# item_count = 0
# while item_count < 100:
#   # Scroll down the page
#   driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

#   # Wait for the page to load
#   time.sleep(1)

#   # Get the new height of the page
#   new_height = driver.execute_script("return document.body.scrollHeight")

#   # If the height of the page has not changed, then we have reached the bottom
#   if new_height == last_height:
#     break

#   # Update the last height
#   last_height = new_height
#   try:

#     # Find all of the search results
#     search_results = driver.find_elements(By.CLASS_NAME, "MjjYud")
#   except:
#     pass

#   # Print the titles of the search results
#   for result in search_results:
#     try:
#         url_searched = result.find_element(By.TAG_NAME, "a")
#         url_searched = url_searched.get_attribute("href")
#         if contains_linkedin(url_searched) and url_searched not in scrapped_links:
#           scrapped_links.append(url_searched)
#     except:
#       pass
#     item_count += 1


# # Close the browser window
# driver.quit()

In [5]:
# len(scrapped_links)

In [6]:
# for link in scrapped_links:
#   print(link)

In [7]:
# url = 'https://www.linkedin.com/in/oscar-byiringiro-9baa8313a/'  # Replace with the URL you want to scrape
# response = requests.get(url)
# html = response.text
# soup = BeautifulSoup(html, 'html.parser')
# soup

# # h1_element = soup.find('h1', class_='text-heading-xlarge')
# # if h1_element:
# #     print(h1_element.text)
# # else:
# #     print("No h1 element found on the page.")

In [18]:
url = 'https://www.linkedin.com/in/oscar-byiringiro-9baa8313a/'  # Replace with the URL you want to scrape
# Navigate to the LinkedIn profile
driver.get(url)

# Wait for the page to load (you might need to adjust the time)
driver.implicitly_wait(10)

# Find element by id
# Get the entire HTML page source
page_source = driver.page_source

# Print the HTML source
print(page_source)
# Close the web driver
driver.quit()

<html lang="en"><head>
        <meta name="pageKey" content="auth_wall_desktop_profile">
<!---->        <meta name="locale" content="en_US">
        <meta id="config" data-app-version="2.1.46" data-call-tree-id="AAYJuvjPuARCIZUH1pBHiQ==" data-multiproduct-name="seo-directory-frontend" data-service-name="seo-directory-frontend" data-browser-id="e2779b44-59f7-49f2-860b-b8be6fe83606" data-enable-page-view-heartbeat-tracking="" data-page-instance="urn:li:page:auth_wall_desktop_profile;10g3hL9LQjSAAGCohZSL4g==" data-disable-jsbeacon-pagekey-suffix="false" data-member-id="0" data-msafdf-lib="https://static.licdn.com/aero-v1/sc/h/80ndnja80f2uvg4l8sj2su82m">

        <link rel="canonical" href="/authwall">
<!----><!---->
<!---->
<!---->
<!---->
<!---->
          <link rel="icon" href="https://static.licdn.com/aero-v1/sc/h/al2o9zrvru7aqj8e1x2rzsrca">

        <script>
          function getDfd() {let yFn,nFn;const p=new Promise(function(y, n){yFn=y;nFn=n;});p.resolve=yFn;p.reject=nFn;return p;}